## Dekoratoriai

Python koduose dažnai prireikia modifikuoti funkcijų arba klasių metodų elgesį. Tai gali nutikti, kai norime pakeisti tam tikras detales aprašomos klasės prototipe. Tokiu atveju norėtųsi keisti kuo mažiau kodo. Šiam tikslui yra numatytas įrankis, vadinamas dekoratoriumi. 

Dekoratorius - tai funkcija, kuri pakeičia įvestos funkcijos veikimą ir grąžina rezultatą - pakeistą funkciją. Šiame pavyzdyje naudosime funkciją `modify_serve`, kuri pakeičia kitos funkcijos funkcijos `serve_thing` elgesį:

In [285]:
def modify_serve(func):
    def inner(*args, **kwargs):
        func(*args, **kwargs)
        print("And can you add some sugar on top of it?")
    return inner

def serve_thing(thing):
    print(f"I'd like one {thing} please")

Štai taip veikia funkcija `serve_thing`:

In [287]:
serve_thing('Capuccino')

I'd like one Capuccino please


Taip veiks funkcija `serve_thing`, jei ją dekoruosime su dekoratoriumi `modify_serve`:

In [289]:
fancy_serve = modify_serve(serve_thing)
fancy_serve('Capuccino')

I'd like one Capuccino please
And can you add some sugar on top of it?


Dekoratoriams žymėti Python naudojamas specialus @ simbolis, naudojamas virš dekoruojamos funkcijos apibrėžimo:

In [290]:
@modify_serve
def serve_thing(thing):
    print(f"I'd like one {thing} please")

Tada galime matyti, kad funkcijos `serve_thin` elgesys bus pakeistas:

In [291]:
serve_thing('Cappucino')

I'd like one Cappucino please
And can you add some sugar on top of it?


## `@property`

Grįžkime prie anksčiau turėtos funkcijos `property`. Kadangi ji yra įtaisytoji (builtin), tai įtaisytasis yra ir dekoratorius, žymimas `@`. Jei klasė turi daug atributų, tai yra nepatogu aprašant kiekvieną iš jų vartoti tuos pačius pavadinimus, prasidedančius `get` ir `set`. Tam pasitarnauja dekoratoriai `@property`, `@laipsniai.setter`, `@laipsniai.deleter`. Anksčiau turėtą klasę `Daugianaris` buvo galima aprašyti paprasčiau: 

In [58]:
import numpy as np
class Daugianaris:
    def __init__(self, koeficientai, laipsniai): # apibrėžiamas metodas __init__
        self._laipsniai = laipsniai # apibrėžiamas privatus atributas 
        self._koeficientai = koeficientai # apibrėžiamas privatus atributas
       
    @property
    def laipsniai(self):
        print('Gaunamas atributas Daugianaris._laipsniai')
        return self._laipsniai
      
    @laipsniai.setter
    def laipsniai(self, laipsniai):
        if type(laipsniai) in (np.ndarray, tuple, list): #tikriname ar tipas geras
            if np.all(np.array(laipsniai) % 1 == 0): #tikriname, ar įvesti laipsniai yra sveikieji
                if np.all(np.array(laipsniai) >= 0): #tikriname, ar įvesti laipsniai yra teigiami
                    if len(laipsniai) == len(self._koeficientai): #tikriname, ar laipsnių yra tiek, kiek koeficientų
                        print(f'Laipsnių priskyrimas sėkmingas: {self.__class__.__name__}.laipsniai = {laipsniai}')
                        self._laipsniai = laipsniai
                    else: 
                        raise ValueError('Laipsnių turi būti tiek, kiek koeficientų')
                else:
                    raise ValueError('Laipsnių rodikliai turi būti neneigiami')
            else:
                raise ValueError('Laipsnių rodikliai turi būti sveiki skaičiai')
        else:
            raise TypeError('Laipsniai turi būti np.ndarray, list arba tuple tipo')

    @laipsniai.deleter
    def laipsniai(self):
        print('Ištrinamas atributas Daugianaris._laipsniai')
        del self._laipsniai

Pateiksime taip pat, kaip naudotis šiais trimis `getter`, `setter` ir `deleter` metodais:


In [59]:
d = Daugianaris([1, 3, 3, 1], [0, 1, 2, 3])

In [60]:
print(d.laipsniai) #getter metodas

Gaunamas atributas Daugianaris._laipsniai
[0, 1, 2, 3]


In [61]:
d.laipsniai = [0, 1, 2] #setter metodas

ValueError: Laipsnių turi būti tiek, kiek koeficientų

In [62]:
d.laipsniai = [0, 1, 2, 4] #setter metodas

Laipsnių priskyrimas sėkmingas: Daugianaris.laipsniai = [0, 1, 2, 4]


In [63]:
d.laipsniai

Gaunamas atributas Daugianaris._laipsniai


[0, 1, 2, 4]

In [64]:
del d.laipsniai #deleter

Ištrinamas atributas Daugianaris._laipsniai


## `@staticmethod`

Įprastiniai metodai klasėse reikalauja dirbti su egzemplioriumi (`self`), tačiau tai ne visada patogu. Statiniai metodai - tai tokie metodai, kurie gali egzistuoti klasės apraše nepriklausomai nuo aprašomų objektų (`self`).

Tarkime, kad norime sukurti metodą, kuris suprastina panašiuosius narius daugianaryje. Pavyzdžiui daugianaris $3x^2+ 2x^2 - 7x^3 +11x - 9x+ x^2 - 3$ turi koeficientus `[3, 2, -7, 11, -9, 1, -3]` prie laipsnių `[2, 2, 3, 1, 1, 2, 0]`. Mums reikėtų jį pateikti kaip daugianarį $6x^2 - 7x^3 + 2x - 3$ nebūtinai tokia pačia narių tvarka. 

Norint rasti, kaip pasikeičia koeficientai ir atitinkami laipsniai, reikėtų spręsti bendresnį uždavinį: apskaičiuoti bendrą koeficientų sumą kiekvienai vienodų laipsnių grupei. Tai yra populiarus uždavinys duomenų moksle, todėl norėtųsi, kad jo sprendimo metodas būtų prieinamas už klasės `Daugianaris` ribų.

Tegu daugianario atributai yra tokie:

In [18]:
keys = np.array([3, 2, -7, 11, -9, 1, -3])
values = np.array([2, 2, 3, 1, 1, 2, 0])

Tada grupavimo uždavinį galima spręsti taip:

In [65]:
argidx = np.argsort(values)
keys_sort, values_sort = keys[argidx], values[argidx]
div_points = np.r_[0, np.flatnonzero(np.diff(values_sort)) + 1]
np.add.reduceat(keys_sort, div_points), values_sort[div_points]

(array([-3,  2,  6, -7]), array([0, 1, 2, 3]))

Gautas atsakymas atitinka tai, ko ieškojome. Todėl galime šį metodą panaudoti kaip statinį klasėje `Daugianaris`:

In [67]:
class Daugianaris:
    def __init__(self, koeficientai, laipsniai): # apibrėžiamas metodas __init__
        self._laipsniai = laipsniai # apibrėžiamas privatus atributas 
        self._koeficientai = koeficientai # apibrėžiamas privatus atributas
       
    @staticmethod
    def prastinti(keys, values):
        argidx = np.argsort(values)
        keys_sort, values_sort = keys[argidx], values[argidx]
        div_points = np.r_[0, np.flatnonzero(np.diff(values_sort)) + 1]
        return np.add.reduceat(keys_sort, div_points), values_sort[div_points]

Statinio metodo vienas iš didžiausių privalumų yra, kad jais galima naudoti nesukuriant jokio egzemplioriaus. Prastinimo metodą galima atlikti šitaip:

In [68]:
Daugianaris.prastinti(np.array([3, 2, -7, 11, -9, 1, -3]), np.array([2, 2, 3, 1, 1, 2, 0]))

(array([-3,  2,  6, -7]), array([0, 1, 2, 3]))

## `@njit`
Šis dekoratorius yra bibliotekos `numba` metodas, skirtas pagreitinti kodo veikimą, kompiliuojant kodą ne Python režime, o verčiant jį į optimizuotą mašininį kodą naudojant LLVM kompiliavimo biblioteką. Taip pat yra galimybė kodą vykdyti paraleliai (ant kelių branduolių)

Tarkime yra duotas masyvas, sudarytas iš 10000000 narių. Kiekvienam nariui reikia rasti, ar jis nelygus -2147483648. 

In [71]:
X = np.random.choice(np.array([-2147483648, 0]), p=[0.75, 0.25], size=(10000000,))
idx = X == 0
X[idx] = np.random.randint(7000000, 8000000, size=np.sum(idx))
X

array([-2147483648, -2147483648, -2147483648, ..., -2147483648,
       -2147483648,     7215752])

Palyginame skirtingus būdus tai padaryti laiko atžvilgiu:

In [75]:
%%timeit #efektyviausias numpy metodas
X != -2147483648

7.26 ms ± 171 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [74]:
%%timeit #numpy masyvo iteracija
[n != -2147483648 for n in X]

1.78 s ± 60.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [76]:
%%timeit #numpy masyvo iteracija prieš tai jį pavertus į sąrašą
[n != -2147483648 for n in X.tolist()]

838 ms ± 24.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [77]:
%%timeit #atminties rezervavimas True/False reikšmėms ir jų priskyrimas
r = np.empty(len(X), dtype=bool)
for i in range(len(X)):
    r[i] =  X[i] != -2147483648

2.67 s ± 6.42 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Pastarąjį sprendimo metodą galima perrašyti naudojant `@njit` dekoratorių tokiu būdu:

In [89]:
from numba import njit, prange

@njit
def _numba_mask(r, arr):
    for i in range(len(arr)):
        r[i] = arr[i] != -2147483648
        
def numba_mask(arr):
    r = np.empty(len(arr), dtype=bool)
    _numba_mask(r, arr)
    return r

Tuomet `numpy` masyvo iteracija yra kompiliuojama mašininio kodo lygyje ir pagreitėja tiek, kad susilygina su gerai išoptimizuota operacija 

`X != -2147483648`:

In [86]:
%%timeit
numba_mask(X)

7.68 ms ± 221 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Šį metodą galima dar labiau paspartinti, jei ciklą pagalbiniame `_numba_mask` metode padalinsime keliems branduoliams:

In [103]:
@njit(parallel=True)
def _numba_mask(r, arr):
    for i in prange(len(arr)):
        r[i] = arr[i] != -2147483648

In [104]:
%%timeit
numba_mask(X)

4.07 ms ± 89.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Kai kurie `numpy` metodai naudojant `njit` dekoratorių yra išoptimizuoti dar labiau. Šiuo atveju sparčiausias variantas būtų toks:

In [105]:
@njit(parallel=True, fastmath=True)
def numba_mask(arr):
    return arr != -2147483648

In [106]:
%%timeit
numba_mask(X)

3.92 ms ± 49.6 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


Linux operacinėje sistemoje greitį gavome panašų, tačiau Windows greitis gali skirtis 1.5 karto

## Pastabos

`@staticmethod` ir `@property` yra dekoratoriai, vieni iš dažniausiai pasirodančių klasėse, o `@njit` yra importuojamas išimtinai tik iš `numba` bibliotekos. Pačiam rašyti dekoratorius galima, bet dažniausiai neprireikia.